In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

def normalize_data(data):
    data -= np.mean(data)
    data /= np.max(np.abs(data))
    return data

def create_train_and_test_data(trainId, testId, labels, data):
    test_data = normalize_data(data[testId,:,:,:]).astype(np.float32)
    test_labels = labels[testId]
    train_data = normalize_data(data[trainId,:,:,:]).astype(np.float32)
    train_labels = labels[trainId]
    return train_data, train_labels, test_data, test_labels
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1)==np.argmax(labels,1)))/predictions.shape[0]
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot
def combine_data(data1,data2,num_channels=2):
    data1 = data1.reshape((data1.shape[0],data1.shape[1],data1.shape[2],1))
    data2 = data2.reshape((data2.shape[0],data2.shape[1],data2.shape[2],1))
    combined_data = np.zeros((data1.shape[0],data1.shape[1],data1.shape[2],data1.shape[3]*num_channels))
    combined_data[:,:,:,0] = normalize_data(data1[:,:,:,0])
    combined_data[:,:,:,1] = normalize_data(data2[:,:,:,0])
    return combined_data
def randomize_data(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:,:]
    shuffled_labels = labels[permutation,:]
    return shuffled_dataset, shuffled_labels

numROI = 116
num_channels = 2
num_labels = 2
image_size = numROI
patch_size = image_size
depth = 64
num_hidden = 96
keep_pr = 0.6

In [2]:
clinical = pd.read_csv('D:/mmdps/ER/clinical.csv')
y = clinical['Y'].values
y_hot = dense_to_one_hot(y, num_classes=2)
data1 = np.load('D:/mmdps/ER/BCI_boldnet.npy')
data2 = np.load('D:/mmdps/ER/BCI_dtinet.npy')
combined_data = combine_data(data1, data2)

In [3]:
data1 = data1.reshape((data1.shape[0],data1.shape[1],data1.shape[2],1))
data2 = data2.reshape((data2.shape[0],data2.shape[1],data2.shape[2],1))
data1_norm = normalize_data(data1)
data2_norm = normalize_data(data2)

In [4]:
np.sum(np.isnan(data2))

0

In [5]:
batch_size = 4

In [6]:
num_folds = 10
kf = KFold(n_splits=num_folds,shuffle=True,random_state=42)

In [7]:
K = num_labels
n_batches = len(y) // batch_size
def KL(alpha):
    beta=tf.constant(np.ones((1,K)),dtype=tf.float32)
    S_alpha = tf.reduce_sum(alpha,axis=1,keepdims=True)
    S_beta = tf.reduce_sum(beta,axis=1,keepdims=True)
    lnB = tf.lgamma(S_alpha) - tf.reduce_sum(tf.lgamma(alpha),axis=1,keepdims=True)
    lnB_uni = tf.reduce_sum(tf.lgamma(beta),axis=1,keepdims=True) - tf.lgamma(S_beta)
    
    dg0 = tf.digamma(S_alpha)
    dg1 = tf.digamma(alpha)
    
    kl = tf.reduce_sum((alpha - beta)*(dg1-dg0),axis=1,keepdims=True) + lnB + lnB_uni
    return kl

def mse_loss(p, alpha, global_step, annealing_step): 
    S = tf.reduce_sum(alpha, axis=1, keepdims=True) 
    E = alpha - 1
    m = alpha / S
  
    A = tf.reduce_sum((p-m)**2, axis=1, keepdims=True) 
    B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keepdims=True) 
    
    annealing_coef = tf.minimum(1.0,tf.cast(global_step/annealing_step,tf.float32))
    
    alp = E*(1-p) + 1 
    C =  annealing_coef * KL(alp)
    return (A + B) + C
def loss_EDL(func=tf.digamma):
    def loss_func(p, alpha, global_step, annealing_step): 
        S = tf.reduce_sum(alpha, axis=1, keepdims=True) 
        E = alpha - 1
    
        A = tf.reduce_sum(p * (func(S) - func(alpha)), axis=1, keepdims=True)
    
        annealing_coef = tf.minimum(1.0, tf.cast(global_step/annealing_step,tf.float32))
    
        alp = E*(1-p) + 1 
        B =  annealing_coef * KL(alp)
    
        return (A + B)
    return loss_func

## Softmax

In [8]:
def Softmax_train_test(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        print(test_id)
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, num_channels, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
        
        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
        
            #calculate loss-function (cross-entropy) in training
            logits = model(tf_train_dataset,keep_pr)
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits_v2(labels = tf_train_labels, logits = logits))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * lmb
            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = tf.nn.softmax(logits)
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_prediction = tf.nn.softmax(model(tf_test_dataset,1))
            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                _, l, predictions = session.run(
                [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
                if l < 0.0005 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                # at every 2000. step give some feedback on the progress
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
            
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI softmax Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("Softmax %s.npz" % name, labels=l,predictions=p)


In [10]:
Softmax_train_test(data1_norm, kf, y, name='BOLD',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

[15 16 19 26]
Initialized
Minibatch loss at step 0: 0.737728
Minibatch accuracy: 75.0%
Minibatch loss at step 3479: 0.000498
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
[ 4 12 27 37]
Initialized
Minibatch loss at step 0: 0.723169
Minibatch accuracy: 75.0%
Minibatch loss at step 3415: 0.000499
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 6  9 25 39]
Initialized
Minibatch loss at step 0: 0.768618
Minibatch accuracy: 50.0%
Minibatch loss at step 3986: 0.000499
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
[ 8 13 31 34]
Initialized
Minibatch loss at step 0: 0.749170
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.000543
Minibatch accuracy: 100.0%
Minibatch loss at step 4066: 0.000497
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 0 17 24 33]
Initialized
Minibatch loss at step 0: 0.748372
Minibatch accuracy: 50.0%
Minibatch loss at step 3632: 0.000496
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 1  5 11 29]
Initia

In [11]:
Softmax_train_test(data2_norm, kf, y, name='DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

[15 16 19 26]
Initialized
Minibatch loss at step 0: 0.760889
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.002300
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.000673
Minibatch accuracy: 100.0%
Minibatch loss at step 9076: 0.000498
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 4 12 27 37]
Initialized
Minibatch loss at step 0: 0.793267
Minibatch accuracy: 0.0%
Minibatch loss at step 4000: 0.002044
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.000574
Minibatch accuracy: 100.0%
Minibatch loss at step 8326: 0.000500
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
[ 6  9 25 39]
Initialized
Minibatch loss at step 0: 0.756650
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.002075
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.000738
Minibatch accuracy: 100.0%
Minibatch loss at step 8620: 0.000494
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 8 13 31 34]
Initialized
Minibatch loss at step 0: 0.755615
M

In [12]:
Softmax_train_test(combined_data, kf, y, name='BOLD+DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

[15 16 19 26]
Initialized
Minibatch loss at step 0: 0.727613
Minibatch accuracy: 75.0%
Minibatch loss at step 3265: 0.000497
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
[ 4 12 27 37]
Initialized
Minibatch loss at step 0: 0.721472
Minibatch accuracy: 75.0%
Minibatch loss at step 2975: 0.000499
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
[ 6  9 25 39]
Initialized
Minibatch loss at step 0: 0.760638
Minibatch accuracy: 50.0%
Minibatch loss at step 3138: 0.000500
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 8 13 31 34]
Initialized
Minibatch loss at step 0: 0.794373
Minibatch accuracy: 25.0%
Minibatch loss at step 3316: 0.000497
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 0 17 24 33]
Initialized
Minibatch loss at step 0: 0.731595
Minibatch accuracy: 50.0%
Minibatch loss at step 3575: 0.000499
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
[ 1  5 11 29]
Initialized
Minibatch loss at step 0: 0.710293
Minibatch accuracy: 75.0

## EDL 1

In [12]:
def EDL1_train_test(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
            annealing_step = tf.placeholder(dtype=tf.int32)
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, num_channels, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
            
        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
        
            logits = model(tf_train_dataset,keep_pr)
            def edl1(logits):
                e = tf.nn.relu(logits)
                b = tf.nn.softmax(logits)
                E = tf.reduce_sum(e, axis=1, keepdims=True)
                alpha = E * b+1
                prob = alpha/tf.reduce_sum(alpha, 1, keepdims=True)   
                return prob, alpha
            prob, alpha = edl1(logits)
            loss = tf.reduce_mean(mse_loss(tf_train_labels, alpha, global_step, annealing_step))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * lmb
            
            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = prob
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_prediction, _ = edl1(model(tf_test_dataset,1))
            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, annealing_step : 10* n_batches }
                _, l, predictions = session.run(
                    [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
               
                if l < 0.0005 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                # at every 2000. step give some feedback on the progress
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
            
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI EDL Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("EDL1 %s.npz" % name, labels=l, predictions=p)

In [13]:
EDL1_train_test(data1_norm, kf, y, name='BOLD',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')


Initialized
Minibatch loss at step 0: 0.721999
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.012493
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.005952
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002925
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.014805
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000964
Minibatch accuracy: 100.0%
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.720596
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.014247
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.004961
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002374
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.001241
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000745
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.722993
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.011730
Minibatch accuracy: 100.0%
Minibatch loss at 

In [15]:
EDL1_train_test(data2_norm, kf, y, name='DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')


Initialized
Minibatch loss at step 0: 0.723220
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.014929
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.007952
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.005627
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.004111
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.002336
Minibatch accuracy: 100.0%
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.722242
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.014048
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.006978
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.004450
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.002925
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.001976
Minibatch accuracy: 100.0%
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.722373
Minibatch accuracy: 0.0%
Minibatch loss at step 4000: 0.015224
Minibatch accuracy: 100.0%
Minibatch loss at s

In [16]:
EDL1_train_test(combined_data, kf, y, name='BOLD+DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')


Initialized
Minibatch loss at step 0: 0.714565
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.009615
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.003815
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002048
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.001206
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000699
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.722565
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.013803
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.005103
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002211
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.001176
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000707
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.720385
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.206117
Minibatch accuracy: 75.0%
Minibatch loss at s

## EDL2

In [13]:
def EDL2_train_test(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
            annealing_step = tf.placeholder(dtype=tf.int32)
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, num_channels, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
            evidence1_weight = tf.get_variable('evidence1_weight', shape = [1])
            evidence2_weight = tf.get_variable('evidence2_weight', shape = [1])
        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
        
            logits = model(tf_train_dataset,keep_pr)
            def BMF(logits):
                evidence = tf.nn.relu(logits)
                alpha = evidence + 1
                K = 2
                u = K / tf.reduce_sum(alpha, axis=1, keepdims=True) #uncertainty
                prob = alpha/tf.reduce_sum(alpha, 1, keepdims=True) 
                return prob, u, alpha
            prob, u, alpha = BMF(logits)    
            loss = tf.reduce_mean(mse_loss(tf_train_labels, alpha, global_step, annealing_step))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * lmb
            
            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = prob
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_prediction, test_u, _ = BMF(model(tf_test_dataset,1))
            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, annealing_step : 10* n_batches }
                _, l, predictions = session.run(
                    [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
               
                if l < 0.0005 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                # at every 2000. step give some feedback on the progress
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
            
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI EDL2 Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("EDL2 %s.npz" % name, labels=l, predictions=p)

In [14]:
EDL2_train_test(data1_norm, kf, y, name='BOLD',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.729131
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.004727
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.001741
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.001212
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.000853
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000771
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.739604
Minibatch accuracy: 0.0%
Minibatch loss at step 4000: 0.005581
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.016125
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.001016
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.000820
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000636
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.749505
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.005016
Minibatch accuracy: 100.0%
Minibatch loss at s

In [17]:
EDL2_train_test(data2_norm, kf, y, name='DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.725335
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.013257
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.005916
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.003783
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.002570
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.002232
Minibatch accuracy: 100.0%
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.718126
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.012271
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.005307
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.014449
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.010413
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.001828
Minibatch accuracy: 100.0%
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.727666
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.012252
Minibatch accuracy: 100.0%
Minibatch loss at 

In [16]:
EDL2_train_test(combined_data, kf, y, name='BOLD+DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.737706
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.003943
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.001515
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.000722
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.000585
Minibatch accuracy: 100.0%
Minibatch loss at step 16356: 0.000493
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.719328
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.003633
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.001174
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.000688
Minibatch accuracy: 100.0%
Minibatch loss at step 14653: 0.000498
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.716928
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.003720
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.001253
Minibatch accuracy: 100.0%

## EDL1_fuse

In [21]:
def EDL1_fuse(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            annealing_step = tf.placeholder(dtype=tf.int32) 
            global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
            
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, 1, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
            evidence1_weight = tf.get_variable('evidence1_weight', shape = [1])
            evidence2_weight = tf.get_variable('evidence2_weight', shape = [1])
        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
            train_shape = [tf_train_dataset.shape[0], tf_train_dataset.shape[1], tf_train_dataset.shape[2],1]
            logits1 = model(tf.reshape(tf_train_dataset[:,:,:,0], train_shape),keep_pr)
            logits2 = model(tf.reshape(tf_train_dataset[:,:,:,1], train_shape),keep_pr)
            def edl1_combine(logits1, logits2):
                e1 = tf.nn.relu(logits1)
                b1 = tf.nn.softmax(logits1)
                alpha1= e1
                S1 = tf.reduce_sum(alpha1, axis=1, keepdims=True)

                e2 = tf.nn.relu(logits2)
                b2 = tf.nn.softmax(logits2)
                alpha2= e2
                S2 = tf.reduce_sum(alpha2, axis=1, keepdims=True)

                S = (S1+S2)/2
                b = b1*b2#evidence1_weight*b1*evidence2_weight*b2
                kappa = tf.reduce_sum(b, axis=1,keepdims=True)
                b_new = b/kappa
                alpha_new = S*b_new+1
                prob = alpha_new / tf.reduce_sum(alpha_new, axis=1,keepdims=True) 
                return prob, alpha_new
            prob, alpha_new = edl1_combine(logits1=logits1,logits2=logits2)        
            loss = tf.reduce_mean(mse_loss(tf_train_labels, alpha_new, global_step, annealing_step))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * 0.001

            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = prob
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_shape = [tf_test_dataset.shape[0], tf_test_dataset.shape[1], tf_test_dataset.shape[2],1]
            test_prediction,_ = edl1_combine(model(tf.reshape(tf_test_dataset[:,:,:,0], test_shape),1),
                                            model(tf.reshape(tf_test_dataset[:,:,:,1], test_shape),1))
            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, annealing_step : 10* n_batches }
                _, l, predictions = session.run(
                    [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
               
                if l < 0.001 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                # at every 2000. step give some feedback on the progress
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI EDL1_fuse Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("EDL1_fuse %s.npz" % name, labels=l,predictions=p)

In [22]:
EDL1_fuse(combined_data, kf, y, name='BOLD_combinedDTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.719367
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.012067
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.006270
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.003561
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.002274
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.001492
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.722537
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.011868
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.491873
Minibatch accuracy: 75.0%
Minibatch loss at step 12000: 0.003518
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.002339
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.001675
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.719168
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.013455
Minibatch accuracy: 100.0%
Minibatch loss at s

## EDL2_fuse

In [28]:
def EDL2_fuse(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
            annealing_step = tf.placeholder(dtype=tf.int32)
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, 1, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
            evidence1_weight = tf.get_variable('evidence1_weight', shape = [1])
            evidence2_weight = tf.get_variable('evidence2_weight', shape = [1])

        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
            
            train_shape = [tf_train_dataset.shape[0], tf_train_dataset.shape[1], tf_train_dataset.shape[2],1]
            logits1 = model(tf.reshape(tf_train_dataset[:,:,:,0], train_shape),keep_pr)
            logits2 = model(tf.reshape(tf_train_dataset[:,:,:,1], train_shape),keep_pr)
            k=2
            def edl2_combine(logits1, logits2):
                e1 = tf.nn.relu(logits1)
                alpha1 = e1 + 1
                S1 = tf.reduce_sum(alpha1, axis=1, keepdims=True)
                u1 = k/S1
                b1 = e1/S1
                e2 = tf.nn.relu(logits2)
                alpha2 = e2 + 1
                S2 = tf.reduce_sum(alpha2, axis=1, keepdims=True)
                u2 = k/S2
                b2 = e2/S2

                b = b1*b2 + b1*u2 + b2*u1
                u = u1*u2
                kappa = tf.reduce_sum(b,axis=1,keepdims=True)+u
                b_new = b/kappa
                u_new = u/kappa
                S = k/u_new
                alpha_new = S*b_new+1
                prob = alpha_new / tf.reduce_sum(alpha_new, axis=1, keepdims=True)
                return prob, u, alpha_new

            prob, u, alpha_new = edl2_combine(logits1, logits2)

            loss = tf.reduce_mean(mse_loss(tf_train_labels, alpha_new, global_step, annealing_step))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * lmb
            
            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = prob
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_shape = [tf_test_dataset.shape[0], tf_test_dataset.shape[1], tf_test_dataset.shape[2],1]
            test_prediction, test_u, _ = edl2_combine(model(tf.reshape(tf_test_dataset[:,:,:,0], test_shape),1),
                                                       model(tf.reshape(tf_test_dataset[:,:,:,1], test_shape),1))            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, annealing_step : 10* n_batches }
                _, l, predictions = session.run(
                    [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
               
                
                # at every 2000. step give some feedback on the progress
                if l < 0.001 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))

            
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI EDL2_fuse Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("EDL2_fuse %s.npz" % name, labels=l,predictions=p)

In [29]:
EDL2_fuse(combined_data, kf, y, name='BOLD_combinedDTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')


Initialized
Minibatch loss at step 0: 0.716227
Minibatch accuracy: 25.0%
Minibatch loss at step 2822: 0.000989
Minibatch accuracy: 100.0%
Converged
Test accuracy: 25.0%
Initialized
Minibatch loss at step 0: 0.720585
Minibatch accuracy: 50.0%
Minibatch loss at step 2630: 0.000985
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.723585
Minibatch accuracy: 25.0%
Minibatch loss at step 2771: 0.000999
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.716653
Minibatch accuracy: 25.0%
Minibatch loss at step 3091: 0.000996
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.694048
Minibatch accuracy: 75.0%
Minibatch loss at step 2846: 0.000999
Minibatch accuracy: 100.0%
Converged
Test accuracy: 100.0%
Initialized
Minibatch loss at step 0: 0.695257
Minibatch accuracy: 50.0%
Minibatch loss at step 2904: 0.000998
Minibatch accuracy: 100.0%
Converged
Test a

## EDL1+EDL2

In [30]:
def EDL12(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
            annealing_step = tf.placeholder(dtype=tf.int32)
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, num_channels, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
            
        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
        
            logits = model(tf_train_dataset,keep_pr)
            
            def edl12(logits):
                e = tf.nn.relu(logits)
                b2 = tf.nn.softmax(logits)

                alpha1 = e + 1
                S = tf.reduce_sum(alpha1,axis=1,keepdims=True)
                b1 = e / S
                u1 = num_labels/S
                b = b1*b2+u1*b2

                kappa = tf.reduce_sum(b,axis=1,keepdims=True)
                b_new = (1-u1)*b/kappa
                alpha_new = S*b_new+1
                prob = alpha_new/tf.reduce_sum(alpha_new, 1, keepdims=True)   
                return prob, alpha_new
            prob, alpha = edl12(logits)
            loss = tf.reduce_mean(mse_loss(tf_train_labels, alpha, global_step, annealing_step))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * lmb
            
            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = prob
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_prediction, _ = edl12(model(tf_test_dataset,1))
            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, annealing_step : 10* n_batches }
                _, l, predictions = session.run(
                    [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
               
                if l < 0.0005 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                # at every 2000. step give some feedback on the progress
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
            
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI EDL12 Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("EDL12 %s.npz" % name, labels=l, predictions=p)

In [46]:
EDL12(data1_norm, kf, y, name='BOLD',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.719861
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.011975
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.004807
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002526
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.001268
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.001883
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.717443
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.456241
Minibatch accuracy: 75.0%
Minibatch loss at step 8000: 0.004652
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002393
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.001351
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000794
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.720640
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.010259
Minibatch accuracy: 100.0%
Minibatch loss at s

In [47]:
EDL12(data2_norm, kf, y, name='DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.721094
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.017496
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.009018
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.004869
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.003353
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.006728
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.721438
Minibatch accuracy: 0.0%
Minibatch loss at step 4000: 0.018006
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.007347
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.004511
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.002800
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.001845
Minibatch accuracy: 100.0%
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.721187
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.014740
Minibatch accuracy: 100.0%
Minibatch loss at s

In [49]:
EDL12(combined_data, kf, y, name='BOLD+DTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')


Initialized
Minibatch loss at step 0: 0.723144
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.011353
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.004878
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.002458
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.001314
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000785
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.723136
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.009467
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.003321
Minibatch accuracy: 100.0%
Minibatch loss at step 12000: 0.001949
Minibatch accuracy: 100.0%
Minibatch loss at step 16000: 0.000888
Minibatch accuracy: 100.0%
Minibatch loss at step 20000: 0.000583
Minibatch accuracy: 100.0%
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.714270
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: 0.016947
Minibatch accuracy: 100.0%
Minibatch loss at 

## EDL1+EDL2_fuse

In [56]:
def EDL12_fuse(data, kf, y, name,learning_rate=0.0001, lmb = 0.001, num_steps = 4001, loss_function = None):
    test_labs1=[]
    test_preds1=[]
    num_channels = data.shape[3]
    for train_id, test_id in kf.split(y):
        train_data, train_labels, test_data, test_labels =  create_train_and_test_data(train_id, test_id, y_hot, data)
        graph = tf.Graph()
        with graph.as_default():
        
        #input data placeholders
            tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
            tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
            global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
            annealing_step = tf.placeholder(dtype=tf.int32)
            #test data is a constant
            tf_test_dataset = tf.constant(test_data)
            
            #network weight variables: Xavier initialization for better convergence in deep layers
            layer1_weights = tf.get_variable("layer1_weights", shape=[1, patch_size, 1, depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer1_biases = tf.Variable(tf.constant(0.001, shape=[depth]))
            layer2_weights = tf.get_variable("layer2_weights", shape=[patch_size, 1, depth, 2*depth],
                initializer=tf.contrib.layers.xavier_initializer())
            layer2_biases = tf.Variable(tf.constant(0.001, shape=[2*depth]))
            layer3_weights = tf.get_variable("layer3_weights", shape=[2*depth, num_hidden],
                initializer=tf.contrib.layers.xavier_initializer())
            layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_hidden]))
            layer4_weights = tf.get_variable("layer4_weights", shape=[num_hidden, num_labels],
                initializer=tf.contrib.layers.xavier_initializer())
            layer4_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
            evidence1_weight = tf.get_variable('evidence1_weight', shape = [1])
            evidence2_weight = tf.get_variable('evidence2_weight', shape = [1])

        #convolutional network architecture
            def model(data, keep_pr):
                #first layer: line-by-line convolution with ReLU and dropout
                conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer1_biases),keep_pr)
                #second layer: convolution by column with ReLU and dropout
                conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='VALID')
                hidden = tf.nn.dropout(tf.nn.relu(conv+layer2_biases),keep_pr)
                #third layer: fully connected hidden layer with dropout and ReLU
                shape = hidden.get_shape().as_list()
                reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),keep_pr)
                #fourth (output) layer: fully connected layer with logits as output
                return tf.matmul(hidden, layer4_weights) + layer4_biases
            
            train_shape = [tf_train_dataset.shape[0], tf_train_dataset.shape[1], tf_train_dataset.shape[2],1]
            logits1 = model(tf.reshape(tf_train_dataset[:,:,:,0], train_shape),keep_pr)
            logits2 = model(tf.reshape(tf_train_dataset[:,:,:,1], train_shape),keep_pr)
            def edl12_combine(logits1, logits2):
                e1 = tf.nn.relu(logits1)
                b12 = tf.nn.softmax(logits1)
                alpha11 = e1 + 1
                S1 = tf.reduce_sum(alpha11,axis=1,keepdims=True)
                b11 = e1 / S1
                u1 = num_labels/S1
                b1 = b11*b12+u1*b12
                kappa1 = tf.reduce_sum(b1,axis=1,keepdims=True)
                b_new1 = (1-u1)*b1/kappa1

                e2 = tf.nn.relu(logits2)
                b22 = tf.nn.softmax(logits1)
                alpha21 = e2 + 1
                S2 = tf.reduce_sum(alpha21,axis=1,keepdims=True)
                b21 = e2 / S2
                u2 = num_labels/S2
                b2 = b21*b22+u2*b22
                kappa2 = tf.reduce_sum(b2,axis=1,keepdims=True)
                b_new2 = (1-u2)*b2/kappa2

                b = b_new1*b_new2 + b_new1*u2 + b_new2*u1
                u = u1*u2
                kappa = tf.reduce_sum(b,axis=1,keepdims=True)+u
                b_new = b/kappa
                u_new = u/kappa
                S = num_labels/u_new
                alpha_new = S*b_new+1
                prob = alpha_new / tf.reduce_sum(alpha_new, axis=1, keepdims=True)
                return prob, u_new, alpha_new

            prob, u, alpha_new = edl12_combine(logits1, logits2)

            loss = tf.reduce_mean(mse_loss(tf_train_labels, alpha_new, global_step, annealing_step))
            l2_loss = (tf.nn.l2_loss(layer3_weights)+tf.nn.l2_loss(layer4_weights)) * lmb
            
            if loss_function == None:
                total_loss = loss
            elif loss_function == 'loss+l2_loss':
                total_loss = loss+l2_loss
            #optimizer definition
            #learning_rate = 0.001
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.9, epsilon=1e-08).minimize(total_loss) 
            
            #calculate predictions from training data
            train_prediction = prob
            #calculate predictions from test data (keep_pr of dropout is 1!)
            test_shape = [tf_test_dataset.shape[0], tf_test_dataset.shape[1], tf_test_dataset.shape[2],1]
            test_prediction, test_u, _ = edl12_combine(model(tf.reshape(tf_test_dataset[:,:,:,0], test_shape),1),
                                                       model(tf.reshape(tf_test_dataset[:,:,:,1], test_shape),1))            
            # nuber of iterations
            #num_steps = 4001
        
        #start TensorFlow session
        with tf.Session(graph=graph) as session:
            tf.global_variables_initializer().run()
            print('Initialized')
            for step in range(num_steps):
                
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                
                if (offset == 0 ): #if we seen all train data at least once, re-randomize the order of instances
                    train_data, train_labels = randomize_data(train_data, train_labels)
                
                #create batch    
                batch_data = train_data[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]
                
                #feed batch data to the placeholders
                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, annealing_step : 10* n_batches }
                _, l, predictions = session.run(
                    [optimizer, total_loss, train_prediction], feed_dict=feed_dict)
               
                
                # at every 2000. step give some feedback on the progress
                if l < 0.0005 or l > 5:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
                    print('Converged')
                    break
                if (step % 4000 == 0):
                    print('Minibatch loss at step %d: %f' % (step, l))
                    print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))

            
            #evaluate the trained model on the test data in the given fold
            test_pred=test_prediction.eval()
            print('Test accuracy: %.1f%%' % accuracy(test_pred, test_labels))
            
            #save test predictions and labels of this fold to a list
            test_labs1.append(test_labels)
            test_preds1.append(test_pred)

    l=test_labs1[0]
    p=test_preds1[0]   
    #iterate through the cross-validation folds    
    for i in range(1,num_folds):
        l=np.vstack((l,test_labs1[i]))
        p=np.vstack((p,test_preds1[i]))

    #calculate final accuracy    
    print('ROI EDL12_fuse Test accuracy: %s, %.2f%%' % (name, accuracy(p, l)))
    np.savez("EDL12_fuse %s.npz" % name, labels=l,predictions=p)

In [57]:
EDL12_fuse(combined_data, kf, y, name='BOLD_combinedDTI',learning_rate=0.0001, lmb = 0.001, num_steps = 20001, loss_function = 'loss+l2_loss')

Initialized
Minibatch loss at step 0: 0.721406
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.004560
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.000981
Minibatch accuracy: 100.0%
Minibatch loss at step 10298: 0.000499
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.723119
Minibatch accuracy: 25.0%
Minibatch loss at step 4000: 0.003827
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.000896
Minibatch accuracy: 100.0%
Minibatch loss at step 9855: 0.000497
Minibatch accuracy: 100.0%
Converged
Test accuracy: 75.0%
Initialized
Minibatch loss at step 0: 0.715850
Minibatch accuracy: 75.0%
Minibatch loss at step 4000: 0.004492
Minibatch accuracy: 100.0%
Minibatch loss at step 8000: 0.001104
Minibatch accuracy: 100.0%
Minibatch loss at step 10937: 0.000499
Minibatch accuracy: 100.0%
Converged
Test accuracy: 50.0%
Initialized
Minibatch loss at step 0: 0.716814
Minibatch accuracy: 50.0%
Minibatch loss at step 4000: